In [1]:
import semantic_kernel as sk
import os
from dotenv import load_dotenv
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion


In [2]:

load_dotenv()
OPENAI_ENDPOINT = os.getenv("OPENAI_USEAST3_ENDPOINT")
OPENAI_API_KEY = os.getenv("OPENAI_USEAST3_API_KEY")



In [6]:
kernel = sk.Kernel()


gpt35 = AzureChatCompletion(deployment_name="gpt-35-turbo", # yours may be different
                            endpoint=OPENAI_ENDPOINT,
                            api_key=OPENAI_API_KEY)

gpt4 = AzureChatCompletion(deployment_name="gpt4", # yours may be different
                            endpoint=OPENAI_ENDPOINT,
                            api_key=OPENAI_API_KEY)

kernel.add_chat_service("gpt35", gpt35)
kernel.add_chat_service("gpt4", gpt4)



In [7]:
prompt = """knock, knock? Who’s there? {{$input}}. {{$input}} who?"""

knock = kernel.create_semantic_function(prompt, temperature=0.8)

response = knock("Dishes")



In [8]:
print(response)

Dishes the police, open up!
